## This notebook inputs the missing values using KNN

In [22]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [31]:
#INPUT, OUTPUT = 'CompleteIndexes.csv','KNNImputed.csv'
INPUT, OUTPUT = 'CompleteWeeklyIndexes.csv', 'KNNWeeklyImputed.csv'

In [32]:
df = pd.read_csv(INPUT, parse_dates=True, index_col=0)
df_test = df.loc['2015-01-02':] #Selecting test data
df = df.loc['2000-01-01':'2015-01-01'] #Selecting training data
df_test.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2015-01-02,2058.199951,2.708700e+09,17.790001,0.0,17832.990234,76270000.0,4230.240234,1.435150e+09,NaN,NaN,...,23721.300781,1.801713e+09,835.969971,333050000.0,NaN,0.60,NaN,NaN,NaN,NaN
2015-01-09,2044.810059,3.364140e+09,17.549999,0.0,17737.369141,93390000.0,4213.279785,1.715830e+09,17197.730469,155200.0,...,23919.949219,2.334810e+09,824.390015,499788100.0,NaN,0.91,NaN,NaN,NaN,NaN
2015-01-16,2019.420044,4.056410e+09,20.950001,0.0,17511.570313,140480000.0,4142.140137,1.970520e+09,16864.160156,171100.0,...,24103.519531,1.716810e+09,862.820007,592856800.0,NaN,0.00,NaN,NaN,NaN,NaN
2015-01-23,2051.820068,3.573560e+09,16.660000,0.0,17672.599609,97110000.0,4278.140137,1.653830e+09,17511.750000,132500.0,...,24850.449219,2.375624e+09,911.729980,804870700.0,NaN,0.00,NaN,NaN,NaN,NaN
2015-01-30,1994.989990,4.568650e+09,20.969999,0.0,17164.949219,168560000.0,4148.430176,2.264230e+09,17674.390625,173500.0,...,24507.050781,1.438942e+09,906.280029,545062800.0,NaN,0.00,NaN,NaN,NaN,NaN


In [33]:
#Scaled data has zero mean and unit variance
scaler = preprocessing.StandardScaler()
scaler.fit(df)
array = scaler.transform(df)
array_test = scaler.transform(df_test)
df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
df_scaled.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-07,0.588334,-1.002211,0.099912,0.0,0.020227,-0.443957,1.707031,-0.537214,1.945019,-1.565826,...,-0.455167,-1.135995,1.241361,-1.046107,NaN,-0.472653,NaN,NaN,NaN,NaN
2000-01-14,0.676196,-1.085193,-0.125931,0.0,0.109104,0.336040,1.914791,-0.494165,2.196016,-1.565826,...,-0.427369,-1.135995,1.432190,-1.046107,NaN,-0.472653,NaN,NaN,NaN,NaN
2000-01-21,0.587926,-1.011385,0.001243,0.0,-0.099881,-0.244602,2.087059,0.035615,2.170210,-1.565826,...,-0.515651,-1.135995,1.289928,-1.046107,NaN,-0.472653,NaN,NaN,NaN,NaN
2000-01-28,0.286643,-1.079295,0.584489,0.0,-0.327300,-0.461189,1.608014,-0.574034,2.353305,-1.565826,...,-0.296374,-1.135995,1.268922,-1.046107,0.239136,-0.472653,NaN,-0.038733,0.104335,1.056953
2000-02-04,0.524886,-1.109498,0.080178,0.0,-0.227555,-0.618274,2.116016,-0.306059,2.461300,-1.565826,...,NaN,NaN,1.664966,-1.046107,NaN,0.309479,NaN,NaN,NaN,NaN


In [34]:
imp = KNNImputer(n_neighbors=5, weights="uniform")
imp.fit(df_scaled)
df_imputed = imp.transform(df_scaled)
df_test_imputed = imp.transform(df_test_scaled)
df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)

In [35]:
df_final.tail()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2014-11-28,2.911367,-0.240038,-0.819907,0.0,2.816486,-1.438162,2.665750,-1.799580,1.703751,0.483228,...,1.291229,1.966740,0.647232,1.115397,-1.028382,-0.472653,0.224111,-1.161981,-0.616267,-0.748109
2014-12-05,2.940345,0.305022,-0.977778,0.0,2.874378,-1.451109,2.634646,-0.275012,1.855242,0.205623,...,1.294320,3.806885,0.708656,1.525285,-0.417633,-0.472653,0.085526,0.502217,-0.276995,-0.496527
2014-12-12,2.669338,0.744673,0.029747,0.0,2.573736,-1.043260,2.501441,-0.033442,1.674719,1.382009,...,1.140995,0.477969,0.360097,1.413037,-0.926569,0.017483,0.224111,0.567321,0.306237,-0.869256
2014-12-19,2.922831,2.119495,-0.473467,0.0,2.806092,1.067770,2.599307,2.742025,1.756885,0.850162,...,1.114018,1.135032,0.528775,3.645977,-0.704534,0.507619,0.342817,0.432828,-0.028040,-0.790725
2014-12-26,2.990064,-0.698382,-0.691636,0.0,2.916471,-1.703778,2.637635,-1.935233,1.821862,-0.377072,...,1.298251,0.552799,0.515408,0.226231,-0.481866,2.186597,0.187499,0.332941,-0.400199,-0.480409


In [36]:
# Descaling the data
array_2 = scaler.inverse_transform(df_final)
df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

array_test_2 = scaler.inverse_transform(df_test_final)
df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)

df_final2.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n255_Close,n255_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-07,1441.469971,1.225200e+09,21.719999,0.0,11522.559570,184900000.0,3529.600098,1.634930e+09,18193.410156,0.0,...,15405.629883,0.0,943.880005,2.980232e-08,26.228083,0.00,67.608411,78.920857,80.800261,0.029950
2000-01-14,1465.150024,1.085900e+09,19.660000,0.0,11722.980469,266830000.0,3704.739990,1.656630e+09,18956.550781,0.0,...,15542.230469,0.0,973.859985,2.980232e-08,26.581681,0.00,55.703358,59.817220,66.518457,0.030678
2000-01-21,1441.359985,1.209800e+09,20.820000,0.0,11251.709961,205840000.0,3849.959961,1.923680e+09,18878.089844,0.0,...,15108.410156,0.0,951.510010,2.980232e-08,24.398514,0.00,67.608411,58.920494,71.545688,0.032724
2000-01-28,1360.160034,1.095800e+09,26.139999,0.0,10738.870117,183090000.0,3446.129883,1.616370e+09,19434.779297,0.0,...,16185.940430,0.0,948.210022,2.980232e-08,24.412899,0.00,67.608411,71.914247,117.577146,0.033573
2000-02-04,1424.369995,1.045100e+09,21.540001,0.0,10963.799805,166590000.0,3874.370117,1.751450e+09,19763.130859,0.0,...,16508.498242,0.0,1010.429993,2.980232e-08,26.421125,0.75,56.185971,58.794812,75.979000,0.027932


In [37]:
output = OUTPUT.split('.')
df_final2.to_csv(OUTPUT, index = True)
df_test_final2.to_csv(output[0]+ '_test.' + output[1], index = True)